In [61]:
!pip install qdrant_client

In [62]:
from qdrant_client import QdrantClient , models

In [63]:
!pip install -U sentence-transformers


In [64]:
import ast

In [65]:
from sentence_transformers import SentenceTransformer

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [67]:
df = pd.read_csv('/content/tmdb_5000_movies.csv' , usecols = ['genres', 'overview' , 'tagline' , 'title' , 'popularity'])
df

,genres,overview,popularity,tagline,title
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...",150.437577,Enter the World of Pandora.,Avatar
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...",139.082615,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,107.376788,A Plan No One Escapes,Spectre
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",Following the death of District Attorney Harve...,112.312950,The Legend Ends,The Dark Knight Rises
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","John Carter is a war-weary, former military ca...",43.926995,"Lost in our world, found in another.",John Carter
...,...,...,...,...,...
4798,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",El Mariachi just wants to play his guitar and ...,14.269792,"He didn't come looking for trouble, but troubl...",El Mariachi
4799,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",A newlywed couple's honeymoon is upended by th...,0.642552,A newlywed couple's honeymoon is upended by th...,Newlyweds
4800,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,NaN,"Signed, Sealed, Delivered"
4801,[],When ambitious New York attorney Sam is sent t...,0.857008,A New Yorker in Shanghai,Shanghai Calling


In [68]:
df['genres'] = df['genres'].apply(lambda x: [genre['name'] for genre in ast.literal_eval(x)])


In [69]:
df = df[['title' ,'genres',	'overview',	'popularity'	,'tagline'	]]

In [70]:
df = df[df['tagline'].notna()]

In [71]:
df


,title,genres,overview,popularity,tagline
0,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...",150.437577,Enter the World of Pandora.
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","Captain Barbossa, long believed to be dead, ha...",139.082615,"At the end of the world, the adventure begins."
2,Spectre,"[Action, Adventure, Crime]",A cryptic message from Bond’s past sends him o...,107.376788,A Plan No One Escapes
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",Following the death of District Attorney Harve...,112.312950,The Legend Ends
4,John Carter,"[Action, Adventure, Science Fiction]","John Carter is a war-weary, former military ca...",43.926995,"Lost in our world, found in another."
...,...,...,...,...,...
4795,Bang,[Drama],A young woman in L.A. is having a bad day: she...,0.918116,Sometimes you've got to break the rules
4796,Primer,"[Science Fiction, Drama, Thriller]",Friends/fledgling entrepreneurs invent a devic...,23.307949,What happens if it actually works?
4798,El Mariachi,"[Action, Crime, Thriller]",El Mariachi just wants to play his guitar and ...,14.269792,"He didn't come looking for trouble, but troubl..."
4799,Newlyweds,"[Comedy, Romance]",A newlywed couple's honeymoon is upended by th...,0.642552,A newlywed couple's honeymoon is upended by th...


In [72]:
data = df.sample(1000).to_dict('records')

In [73]:
data

[{'title': 'The Forsaken',
  'genres': ['Action', 'Adventure', 'Horror', 'Thriller'],
  'overview': 'A young man is in a race against time as he searches for a cure after becoming infected with a virus that will eventually turn him into a blood-sucking vampire.',
  'popularity': 2.739307,
  'tagline': 'The night... has an appetite.'},
 {'title': 'The Object of My Affection',
  'genres': ['Comedy', 'Drama', 'Romance'],
  'overview': "A pregnant New York social worker begins to develop romantic feelings for her gay best friend, and decides she'd rather raise her child with him, much to the dismay of her overbearing boyfriend.",
  'popularity': 5.164281,
  'tagline': "Sometimes The Most Desirable Relationship Is The One You Can't Have"},
 {'title': '[REC]²',
  'genres': ['Thriller', 'Horror'],
  'overview': 'The action continues from [REC], with the medical officer and a SWAT team outfitted with video cameras are sent into the sealed off apartment to control the situation.',
  'popularity

In [74]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # this encoder is used for vecotrization of data you provided

In [75]:
qdrant = QdrantClient(':memory:')

In [76]:
qdrant.recreate_collection(
    collection_name = 'Movies',
    vectors_config = models.VectorParams(
        size = encoder.get_sentence_embedding_dimension(),
        distance = models.Distance.COSINE
    )
)

<ipython-input-76-31f2bcc10134>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [ ]:
qdrant.upload_records(
    collection_name = 'Movies',
    records=[
        models.Record(
            id = idx,
            vector = encoder.encode(doc['overview']).tolist(),
            payload = doc
        )for idx,doc in enumerate(data)
    ]
)

In [ ]:
hits = qdrant.search(
    collection_name = 'Movies',
    query_vector = encoder.encode("lust movie").tolist(),
    limit = 3
)
for hit in hits:
  print(hit.payload , "score" , hit.score)